In [ ]:
# Импорты и настройка
from pathlib import Path
import json
import re
import xml.etree.ElementTree as ET
import csv
import pandas as pd


In [ ]:
class SentenceBasedProcessor:
    """Процессор на основе целых предложений и осмысленных фраз"""
    
    def __init__(self):
        # Паттерны для поиска готовых техник и диалогов
        self.technique_patterns = {
            'выявление_потребностей': [
                r'[А-ЯЁ][^.!?]*(?:что|как|почему|когда|где|какой|какая|какие)[^.!?]*\?',
                r'[А-ЯЁ][^.!?]*(?:расскажите|объясните|опишите|поделитесь)[^.!?]*[.!?]',
                r'[А-ЯЁ][^.!?]*(?:важно|интересует|волнует|беспокоит)[^.!?]*\?',
                r'[А-ЯЁ][^.!?]*(?:потребност|нужд|требован)[^.!?]*[.!?]',
                r'[А-ЯЁ][^.!?]*СПИН[^.!?]*[.!?]'
            ],
            'презентация_объектов': [
                r'[А-ЯЁ][^.!?]*(?:это означает что|благодаря этому|в результате)[^.!?]*[.!?]',
                r'[А-ЯЁ][^.!?]*(?:представьте себе|посмотрите|обратите внимание)[^.!?]*[.!?]',
                r'[А-ЯЁ][^.!?]*(?:выгода|преимущество|польза)[^.!?]*[.!?]',
                r'[А-ЯЁ][^.!?]*(?:квартира|дом|объект)[^.!?]*(?:характеристик|особенност)[^.!?]*[.!?]'
            ],
            'работа_с_возражениями': [
                r'[А-ЯЁ][^.!?]*(?:понимаю|согласен)[^.!?]*(?:но|однако|тем не менее)[^.!?]*[.!?]',
                r'[А-ЯЁ][^.!?]*(?:возражение|сомнение)[^.!?]*[.!?]',
                r'[А-ЯЁ][^.!?]*(?:да но да|бумеранг)[^.!?]*[.!?]',
                r'[А-ЯЁ][^.!?]*дорого[^.!?]*[.!?]'
            ],
            'установление_доверия': [
                r'[А-ЯЁ][^.!?]*(?:опыт показывает|практика показывает)[^.!?]*[.!?]',
                r'[А-ЯЁ][^.!?]*(?:другие клиенты|наши клиенты)[^.!?]*[.!?]',
                r'[А-ЯЁ][^.!?]*(?:гарантирую|обещаю|ручаюсь)[^.!?]*[.!?]',
                r'[А-ЯЁ][^.!?]*(?:доверие|надежность)[^.!?]*[.!?]'
            ],
            'закрытие_сделки': [
                r'[А-ЯЁ][^.!?]*(?:готовы|согласны)[^.!?]*(?:подписать|оформить|купить)[^.!?]*\?',
                r'[А-ЯЁ][^.!?]*(?:когда удобно|когда вам подходит)[^.!?]*\?',
                r'[А-ЯЁ][^.!?]*(?:выбираете|принимаете решение)[^.!?]*\?',
                r'[А-ЯЁ][^.!?]*(?:первый вариант или второй)[^.!?]*\?'
            ]
        }
        
        # Паттерны для поиска диалогов
        self.dialog_patterns = [
            r'(?:Менеджер|Продавец|Риелтор|Агент):\s*([^.!?]+[.!?])',
            r'(?:Клиент|Покупатель|Заказчик):\s*([^.!?]+[.!?])',
            r'—\s*([А-ЯЁ][^.!?]+[.!?])',
            r'«([^»]+)»[^А-ЯЁ]*говорит'
        ]
        
        # Готовые примеры техник (из теории продаж)
        self.theory_examples = {
            'выявление_потребностей': [
                "Что для вас важно при выборе квартиры?",
                "Расскажите о ваших требованиях к жилью.",
                "Какие факторы влияют на ваше решение?",
                "Что должно быть в идеальной квартире?",
                "Как вы представляете свой новый дом?"
            ],
            'презентация_объектов': [
                "Это означает, что вы сэкономите час времени каждый день.",
                "Благодаря этой планировке ваша семья будет чувствовать себя комфортно.",
                "Представьте, как здорово будет встречать рассветы на этом балконе.",
                "Эта особенность дает вам преимущество перед соседями."
            ],
            'работа_с_возражениями': [
                "Понимаю ваши сомнения, давайте разберем детали.",
                "Согласен, цена важна, но посмотрите на выгоды.",
                "Многие клиенты сначала так думают, но потом понимают.",
                "Это важный вопрос, спасибо что подняли его."
            ],
            'установление_доверия': [
                "Мой опыт показывает, что это лучшее решение.",
                "Другие клиенты остались очень довольны.",
                "Гарантирую, что вы не пожалеете о выборе.",
                "Наша компания работает на рынке 15 лет."
            ],
            'закрытие_сделки': [
                "Когда вам удобно подписать договор?",
                "Готовы оформить бронирование сегодня?",
                "Что выбираете - первый вариант или второй?",
                "Когда планируете въезжать?"
            ]
        }


In [ ]:
# Методы для извлечения текста из FB2
def extract_text_clean(self, file_path):
    """Чистое извлечение текста"""
    
    if file_path.suffix.lower() == '.fb2':
        return self.extract_fb2_sentences(file_path)
    else:
        return self.extract_txt_sentences(file_path)

def extract_fb2_sentences(self, fb2_path):
    """Извлечение FB2 с сохранением структуры предложений"""
    
    methods = [
        ('xml_parse', self._xml_extract),
        ('regex_clean', self._regex_extract),
        ('binary_force', self._binary_extract)
    ]
    
    for method_name, method_func in methods:
        try:
            text = method_func(fb2_path)
            if text and len(text) > 1000:
                return self.clean_and_structure_text(text)
        except Exception as e:
            continue
    
    return ""

def _xml_extract(self, fb2_path):
    """XML извлечение"""
    tree = ET.parse(fb2_path)
    root = tree.getroot()
    
    paragraphs = []
    for elem in root.iter():
        if elem.tag and elem.tag.endswith('p') and elem.text:
            text = elem.text.strip()
            if len(text) > 10:
                paragraphs.append(text)
    
    return '\n\n'.join(paragraphs)

def _regex_extract(self, fb2_path):
    """Regex извлечение с попыткой разных кодировок"""
    for encoding in ['utf-8', 'cp1251', 'koi8-r']:
        try:
            with open(fb2_path, 'r', encoding=encoding) as f:
                content = f.read()
            
            # Более умная очистка
            clean_text = re.sub(r'<[^>]*>', '\n', content)
            clean_text = re.sub(r'\n\s*\n', '\n\n', clean_text)
            
            if len(clean_text) > 2000:
                return clean_text
        except:
            continue
    return ""

def _binary_extract(self, fb2_path):
    """Принудительное извлечение"""
    with open(fb2_path, 'rb') as f:
        raw_data = f.read()
    
    for encoding in ['utf-8', 'cp1251']:
        try:
            content = raw_data.decode(encoding, errors='ignore')
            clean_text = re.sub(r'<[^>]*>', '\n', content)
            if len(clean_text) > 2000:
                return clean_text
        except:
            continue
    return ""

def extract_txt_sentences(self, txt_path):
    """Извлечение TXT с разными кодировками"""
    for encoding in ['utf-8', 'cp1251', 'koi8-r', 'windows-1252']:
        try:
            with open(txt_path, 'r', encoding=encoding) as f:
                text = f.read()
            if len(text) > 500:
                return self.clean_and_structure_text(text)
        except UnicodeDecodeError:
            continue
        except Exception as e:
            print(f"⚠️ Ошибка при чтении {txt_path.name}: {e}")
            continue
    return ""

def process_multiple_formats(self, file_path):
    """Обработка разных форматов файлов"""
    file_ext = file_path.suffix.lower()
    
    if file_ext == '.fb2':
        return self.extract_fb2_sentences(file_path)
    elif file_ext == '.txt':
        return self.extract_txt_sentences(file_path)
    elif file_ext in ['.doc', '.docx']:
        return self.extract_doc_text(file_path)
    elif file_ext == '.pdf':
        return self.extract_pdf_text(file_path)
    else:
        print(f"⚠️ Неподдерживаемый формат: {file_ext}")
        return ""

def extract_doc_text(self, doc_path):
    """Извлечение текста из DOC/DOCX файлов"""
    try:
        from docx import Document 
        doc = Document(doc_path)
        text = '\n'.join([paragraph.text for paragraph in doc.paragraphs])
        return self.clean_and_structure_text(text) if len(text) > 500 else ""
    except ImportError:
        print("⚠️ Установите python-docx для работы с .docx файлами")
        return ""
    except Exception as e:
        print(f"⚠️ Ошибка при чтении {doc_path.name}: {e}")
        return ""

def extract_pdf_text(self, pdf_path):
    """Извлечение текста из PDF файлов"""
    try:
        import PyPDF2
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            text = ""
            for page in reader.pages:
                text += page.extract_text() + "\n"
        return self.clean_and_structure_text(text) if len(text) > 500 else ""
    except ImportError:
        print("⚠️ Установите PyPDF2 для работы с .pdf файлами")
        return ""
    except Exception as e:
        print(f"⚠️ Ошибка при чтении {pdf_path.name}: {e}")
        return ""

# Добавляем методы к классу
SentenceBasedProcessor.extract_text_clean = extract_text_clean
SentenceBasedProcessor.extract_fb2_sentences = extract_fb2_sentences
SentenceBasedProcessor.extract_txt_sentences = extract_txt_sentences
SentenceBasedProcessor.process_multiple_formats = process_multiple_formats
SentenceBasedProcessor.extract_doc_text = extract_doc_text
SentenceBasedProcessor.extract_pdf_text = extract_pdf_text
SentenceBasedProcessor._xml_extract = _xml_extract
SentenceBasedProcessor._regex_extract = _regex_extract
SentenceBasedProcessor._binary_extract = _binary_extract


In [ ]:
# Добавляем остальные методы к классу
def clean_and_structure_text(self, text):
    """Очистка и структурирование текста"""
    
    # Удаляем технический мусор
    text = re.sub(r'(?:ISBN|ББК|УДК)[^\n]*', '', text)
    text = re.sub(r'http[^\s]*', '', text)
    text = re.sub(r'\d{4}-\d{4}-\d{4}-\d{4}', '', text)  # номера
    
    # Нормализуем пробелы
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'\n\s*\n', '\n\n', text)
    
    # Убираем короткие строки (мусор)
    lines = text.split('\n')
    clean_lines = []
    for line in lines:
        line = line.strip()
        if (len(line) > 15 and 
            not line.isdigit() and
            not re.match(r'^[A-Z\s]{5,}$', line)):  # не только заглавные
            clean_lines.append(line)
    
    return '\n'.join(clean_lines)

def find_complete_sentences(self, text, book_name):
    """Находит ЦЕЛЫЕ предложения с техниками"""
    
    found_techniques = []
    
    # Разбиваем на предложения
    sentences = re.split(r'[.!?]+\s+', text)
    
    for category, patterns in self.technique_patterns.items():
        category_sentences = []
        
        for pattern in patterns:
            for sentence in sentences:
                sentence = sentence.strip()
                if len(sentence) < 10 or len(sentence) > 300:
                    continue
                
                # Проверяем соответствие паттерну
                if re.search(pattern, sentence, re.IGNORECASE):
                    # Дополнительная проверка качества
                    if self.is_quality_sentence(sentence, category):
                        # Восстанавливаем знак препинания
                        if not sentence[-1] in '.!?':
                            sentence += '.'
                        
                        category_sentences.append({
                            'book': book_name,
                            'category': category,
                            'technique': sentence,
                            'source': 'extracted',
                            'quality': self.rate_sentence_quality(sentence, category)
                        })
        
        # Сортируем по качеству и берем лучшие
        category_sentences.sort(key=lambda x: x['quality'], reverse=True)
        found_techniques.extend(category_sentences[:10])  # Топ-10 на категорию
    
    return found_techniques

def find_dialogs(self, text, book_name):
    """Находит готовые диалоги"""
    
    dialogs = []
    
    for pattern in self.dialog_patterns:
        matches = re.finditer(pattern, text, re.IGNORECASE)
        for match in matches:
            dialog_text = match.group(1).strip()
            if self.is_quality_dialog(dialog_text):
                dialogs.append({
                    'book': book_name,
                    'category': 'диалог_пример',
                    'technique': dialog_text,
                    'source': 'dialog',
                    'quality': 9
                })
    
    return dialogs[:15]

def is_quality_sentence(self, sentence, category):
    """Проверка качества предложения"""
    
    sentence_lower = sentence.lower()
    
    # Базовые проверки
    if any(junk in sentence_lower for junk in ['isbn', 'ббк', 'удк', 'http']):
        return False
    
    # Должно содержать релевантные слова
    relevant_words = {
        'выявление_потребностей': ['клиент', 'вопрос', 'нужно', 'важно', 'требование'],
        'презентация_объектов': ['квартира', 'дом', 'выгода', 'преимущество', 'особенность'],
        'работа_с_возражениями': ['возражение', 'сомнение', 'понимаю', 'согласен'],
        'установление_доверия': ['доверие', 'опыт', 'клиенты', 'гарантия'],
        'закрытие_сделки': ['готовы', 'решение', 'выбор', 'покупка', 'договор']
    }
    
    category_words = relevant_words.get(category, [])
    return any(word in sentence_lower for word in category_words)

def is_quality_dialog(self, dialog_text):
    """Проверка качества диалога"""
    return (len(dialog_text) > 10 and 
            len(dialog_text) < 200 and
            any(word in dialog_text.lower() for word in ['клиент', 'квартир', 'покуп', 'продаж']))

def rate_sentence_quality(self, sentence, category):
    """Оценка качества предложения"""
    score = 5  # базовый
    
    # Бонусы
    if '?' in sentence:
        score += 2
    if any(word in sentence.lower() for word in ['клиент', 'покупатель']):
        score += 1
    if len(sentence.split()) > 5:
        score += 1
    if sentence[0].isupper():
        score += 1
    
    return min(10, score)

def add_theory_examples(self, found_techniques):
    """Добавляет готовые примеры из теории"""
    
    for category, examples in self.theory_examples.items():
        for example in examples:
            found_techniques.append({
                'book': 'theory_base',
                'category': 'базовая_теория',
                'technique': example,
                'source': 'theory',
                'quality': 10
            })
    
    return found_techniques

# Добавляем все методы к классу
SentenceBasedProcessor.clean_and_structure_text = clean_and_structure_text
SentenceBasedProcessor.find_complete_sentences = find_complete_sentences
SentenceBasedProcessor.find_dialogs = find_dialogs
SentenceBasedProcessor.is_quality_sentence = is_quality_sentence
SentenceBasedProcessor.is_quality_dialog = is_quality_dialog
SentenceBasedProcessor.rate_sentence_quality = rate_sentence_quality
SentenceBasedProcessor.add_theory_examples = add_theory_examples


In [ ]:
# Создаем экземпляр процессора и запускаем обработку
processor = SentenceBasedProcessor()

# Обрабатываем все книги (поддерживаем разные форматы)
books_dir = Path("books")
all_files = (list(books_dir.glob("*.fb2")) + 
             list(books_dir.glob("*.txt")) + 
             list(books_dir.glob("*.docx")) + 
             list(books_dir.glob("*.doc")) + 
             list(books_dir.glob("*.pdf")))

print("✨ ОБРАБОТКА НА ОСНОВЕ ЦЕЛЫХ ПРЕДЛОЖЕНИЙ")
print("=" * 60)
print(f"📁 Файлов: {len(all_files)}")
print()

all_techniques = []
book_stats = {}

for file_path in all_files:
    print(f"📖 {file_path.name}")
    
    text = processor.process_multiple_formats(file_path)
    if text:
        # Ищем предложения с техниками
        sentences = processor.find_complete_sentences(text, file_path.name)
        
        # Ищем диалоги
        dialogs = processor.find_dialogs(text, file_path.name)
        
        book_techniques = sentences + dialogs
        all_techniques.extend(book_techniques)
        book_stats[file_path.name] = len(book_techniques)
        
        print(f"   ✅ Найдено: {len(book_techniques)} качественных техник")
    else:
        print(f"   ❌ Не удалось извлечь")
        book_stats[file_path.name] = 0
    
    print()

# Добавляем теоретические примеры
all_techniques = processor.add_theory_examples(all_techniques)

print("=" * 60)
print("🎉 РЕЗУЛЬТАТЫ ОБРАБОТКИ:")
print("=" * 60)
print(f"📚 Обработано книг: {len([b for b, c in book_stats.items() if c > 0])}")
print(f"✨ Всего техник: {len(all_techniques)}")
print(f"\n📖 Техники по книгам:")
for book, count in book_stats.items():
    if count > 0:
        short_name = book[:40] + "..." if len(book) > 40 else book
        print(f"  {short_name}: {count}")


In [ ]:
# Сохраняем результаты в CSV
data_dir = Path("data")
data_dir.mkdir(exist_ok=True)

# CSV с читаемыми техниками
csv_path = data_dir / "quality_techniques.csv"
with open(csv_path, 'w', encoding='utf-8', newline='') as f:
    writer = csv.writer(f, quoting=csv.QUOTE_ALL)
    writer.writerow(['book', 'category', 'source', 'quality', 'sales_technique'])
    for technique in all_techniques:
        book = technique['book']
        category = technique['category']
        source = technique['source']
        quality = technique['quality']
        text = technique['technique'].replace('\n', ' ')
        writer.writerow([book, category, source, quality, text])

print(f"💾 Сохранено в: {csv_path}")
print(f"📊 Всего записей: {len(all_techniques)}")

# Загружаем DataFrame для анализа
df = pd.read_csv(csv_path)
print(f"\n📋 Структура данных:")
print(df.info())
print(f"\n📈 Первые 5 записей:")
df.head()


In [ ]:
## Анализ качества извлеченных техник

# Импорты для визуализации
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# Базовая статистика
print("📊 АНАЛИЗ КАЧЕСТВА ДАННЫХ")
print("=" * 50)

print(f"📚 Общая статистика:")
print(f"   • Всего техник: {len(df)}")
print(f"   • Уникальных книг: {df['book'].nunique()}")
print(f"   • Категорий: {df['category'].nunique()}")
print(f"   • Источников: {df['source'].nunique()}")

print(f"\n🏆 Распределение по качеству:")
quality_counts = df['quality'].value_counts().sort_index()
for quality, count in quality_counts.items():
    percentage = count / len(df) * 100
    print(f"   • Качество {quality}: {count} ({percentage:.1f}%)")

print(f"\n📖 Распределение по категориям:")
category_counts = df['category'].value_counts()
for category, count in category_counts.items():
    percentage = count / len(df) * 100
    print(f"   • {category}: {count} ({percentage:.1f}%)")

print(f"\n📚 Распределение по книгам:")
book_counts = df['book'].value_counts()
for book, count in book_counts.items():
    percentage = count / len(df) * 100
    short_name = book[:35] + "..." if len(book) > 35 else book
    print(f"   • {short_name}: {count} ({percentage:.1f}%)")


In [ ]:
# Визуализация результатов обработки

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('📊 Анализ качества извлеченных техник продаж', fontsize=16, fontweight='bold')

# 1. Распределение по качеству
ax1 = axes[0, 0]
quality_counts.plot(kind='bar', ax=ax1, color='skyblue', edgecolor='navy')
ax1.set_title('🏆 Распределение по качеству')
ax1.set_xlabel('Оценка качества')
ax1.set_ylabel('Количество техник')
ax1.tick_params(axis='x', rotation=0)

# Добавляем значения на столбцы
for i, v in enumerate(quality_counts.values):
    ax1.text(i, v + 0.5, str(v), ha='center', va='bottom')

# 2. Распределение по категориям
ax2 = axes[0, 1]
category_counts.plot(kind='bar', ax=ax2, color='lightgreen', edgecolor='darkgreen')
ax2.set_title('📖 Распределение по категориям')
ax2.set_xlabel('Категория')
ax2.set_ylabel('Количество техник')
ax2.tick_params(axis='x', rotation=45)

# Добавляем значения на столбцы
for i, v in enumerate(category_counts.values):
    ax2.text(i, v + 0.5, str(v), ha='center', va='bottom')

# 3. Распределение по книгам (топ-7)
ax3 = axes[1, 0]
book_counts.head(7).plot(kind='bar', ax=ax3, color='coral', edgecolor='darkred')
ax3.set_title('📚 Топ-7 книг по количеству техник')
ax3.set_xlabel('Книга')
ax3.set_ylabel('Количество техник')
ax3.tick_params(axis='x', rotation=45)

# Сокращаем названия книг для лучшего отображения
labels = [label.get_text()[:20] + '...' if len(label.get_text()) > 20 else label.get_text() 
          for label in ax3.get_xticklabels()]
ax3.set_xticklabels(labels)

# Добавляем значения на столбцы
for i, v in enumerate(book_counts.head(7).values):
    ax3.text(i, v + 0.5, str(v), ha='center', va='bottom')

# 4. Качество по источникам
ax4 = axes[1, 1]
source_quality = df.groupby('source')['quality'].mean()
source_quality.plot(kind='bar', ax=ax4, color='plum', edgecolor='purple')
ax4.set_title('📈 Среднее качество по источникам')
ax4.set_xlabel('Источник')
ax4.set_ylabel('Среднее качество')
ax4.tick_params(axis='x', rotation=0)

# Добавляем значения на столбцы
for i, v in enumerate(source_quality.values):
    ax4.text(i, v + 0.05, f'{v:.1f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()


In [ ]:
# Детальный анализ техник по категориям

print("🔍 ДЕТАЛЬНЫЙ АНАЛИЗ ПО КАТЕГОРИЯМ")
print("=" * 60)

categories = df['category'].unique()

for category in sorted(categories):
    category_df = df[df['category'] == category]
    print(f"\n📋 {category.upper()}")
    print("-" * 40)
    print(f"   • Всего техник: {len(category_df)}")
    print(f"   • Среднее качество: {category_df['quality'].mean():.1f}")
    print(f"   • Мин/Макс качество: {category_df['quality'].min()}/{category_df['quality'].max()}")
    
    # Источники для категории
    sources = category_df['source'].value_counts()
    print(f"   • Источники: {dict(sources)}")
    
    # Топ-3 техники по качеству
    top_techniques = category_df.nlargest(3, 'quality')
    print(f"   • Топ-3 техники:")
    for idx, (_, row) in enumerate(top_techniques.iterrows(), 1):
        technique_preview = row['sales_technique'][:80] + "..." if len(row['sales_technique']) > 80 else row['sales_technique']
        print(f"     {idx}. [{row['quality']}⭐] {technique_preview}")
    print()


In [ ]:
# Анализ длины и качества техник

# Добавляем столбец с длиной техник
df['technique_length'] = df['sales_technique'].str.len()

print("📏 АНАЛИЗ ДЛИНЫ ТЕХНИК")
print("=" * 40)
print(f"   • Средняя длина: {df['technique_length'].mean():.0f} символов")
print(f"   • Медианная длина: {df['technique_length'].median():.0f} символов")
print(f"   • Мин/Макс длина: {df['technique_length'].min()}/{df['technique_length'].max()}")

# Корреляция между длиной и качеством
correlation = df['technique_length'].corr(df['quality'])
print(f"   • Корреляция длина-качество: {correlation:.3f}")

# Визуализация зависимости длины от качества
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.scatter(df['technique_length'], df['quality'], alpha=0.6, color='steelblue')
plt.xlabel('Длина техники (символы)')
plt.ylabel('Качество')
plt.title('📊 Зависимость качества от длины')
plt.grid(True, alpha=0.3)

# Добавляем линию тренда
z = np.polyfit(df['technique_length'], df['quality'], 1)
p = np.poly1d(z)
plt.plot(df['technique_length'], p(df['technique_length']), "r--", alpha=0.8)

plt.subplot(1, 2, 2)
df.boxplot(column='technique_length', by='category', ax=plt.gca())
plt.title('📏 Распределение длины по категориям')
plt.suptitle('')  # Убираем автоматический заголовок
plt.xticks(rotation=45)
plt.ylabel('Длина техники (символы)')

plt.tight_layout()
plt.show()


In [ ]:
# Финальная сводка и сохранение дополнительной статистики

print("📋 ИТОГОВАЯ СВОДКА")
print("=" * 50)

# Создаем сводную статистику
summary_stats = {
    'total_techniques': len(df),
    'books_processed': df['book'].nunique(),
    'categories': df['category'].nunique(),
    'avg_quality': df['quality'].mean(),
    'high_quality_techniques': len(df[df['quality'] >= 9]),
    'avg_technique_length': df['technique_length'].mean(),
    'top_category': category_counts.index[0],
    'top_book': book_counts.index[0] if len(book_counts) > 0 else 'N/A'
}

print(f"✨ Общие метрики:")
print(f"   • Всего техник: {summary_stats['total_techniques']}")
print(f"   • Обработано книг: {summary_stats['books_processed']}")
print(f"   • Категорий техник: {summary_stats['categories']}")
print(f"   • Среднее качество: {summary_stats['avg_quality']:.1f}/10")
print(f"   • Высококачественных техник (≥9): {summary_stats['high_quality_techniques']}")
print(f"   • Средняя длина техники: {summary_stats['avg_technique_length']:.0f} символов")

print(f"\n🏆 Лидеры:")
print(f"   • Самая популярная категория: {summary_stats['top_category']}")
print(f"   • Самая продуктивная книга: {summary_stats['top_book'][:40]}...")

print(f"\n💡 Рекомендации для улучшения:")
low_quality_count = len(df[df['quality'] < 7])
if low_quality_count > 0:
    print(f"   • Пересмотреть {low_quality_count} техник с качеством < 7")

category_imbalance = category_counts.max() / category_counts.min()
if category_imbalance > 3:
    print(f"   • Балансировать категории (дисбаланс {category_imbalance:.1f}x)")

if df['technique_length'].std() > 100:
    print(f"   • Стандартизировать длину техник (разброс {df['technique_length'].std():.0f})")

print(f"\n📁 Созданные файлы:")
print(f"   • {csv_path} - основной датасет")

# Сохраняем дополнительную статистику
summary_path = data_dir / "processing_summary.json"
import json
with open(summary_path, 'w', encoding='utf-8') as f:
    json.dump(summary_stats, f, ensure_ascii=False, indent=2)

print(f"   • {summary_path} - сводная статистика")
print(f"\n✅ Обработка завершена успешно!")
